In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.2 MB/s eta 0:00:00


**트랜포머 아키텍쳐**
1. 인코더
* 입력: 토큰의 시퀀스
* 출력: hidden State 또는 Context

2. 디코더
* 입력: 인코더의 은닉 상태 + 직전까지 생성한 출력 토큰 서브 시퀀스
* 출력: 출력 토큰의 시퀀스를 한번에 하나씩 반복 생성

*디코더의 구성*
* self-attention 레이어
* 인코더 디코더 어텐션 레이어: 현재 단계에서 이후에 생성될 토큰을 참조하지 않도록 한다 == 마스킹
* 피드포워드 레이어: 일반적인 전결합층/위 두개의 과정을 통해 얻은 정보를 바탕으로 출력의 형태를 결정한다


***트랜스포머 아키텍처 토큰 임베딩***
1. 입력 토큰을 one-hot 벡터로 변환한다. 이를 통해 각 토큰은 단 하나의 1값과 나머지는 모두 0 값으로 구성된 벡터로 표현한다.
2. one-hot 벡터를 선형 변환하여 연속 벡터 공간으로 매핑한다. **이를 통해 각 토큰은 연속적인 실수값으로 표현된다.**
3. 포지셔널 인코딩을 적용한다. 이를 통해 시퀀스 내에서 각 토큰의 상대적인 위치 정보를 인코딩할 수 있다. 

이러한 과정을 통해 입력 시퀀스의 각 토큰은 고차원의 연속 벡터로 표현된다. 이 과정에서 사용되는 선형 변환 매트릭스는 모델 학습 과정에서 업데이트되는 학습 가능한 파라미터이다.

***트랜스포머 아키텍처 위치 임베딩***

입력 시퀀스 내에서 각 토큰의 상대적인 위치 정보를 인코딩하는 기법이다. 이는 입력 시퀀스 내에서 각 토큰의 위치 정보를 반영하기 위해 사용된다.

위치 임베딩은 포지셔널 인코딩 방식을 사용한다.



*인코더의 구성*

인코더는 입력 시퀀스를 인코딩하는 부분으로, 다수의 층으로 구성되어있다. 각 층은 다수의 ***멀티 헤드 어텐션과 피드포워드 신경망 레이어***로 구성된다.

인코더의 입력으로는 시퀀스의 각 *토큰에 해당하는 임베딩 벡터*와 *위치 임베딩 벡터*가 결합된 형태의 입력 시퀀스가 사용된다.

이 입력 시퀀스는 각각의 인코더 층을 통과하면서 인코딩되어 최종적으로는 인코딩된 시퀀스가 출력된다.

인코더의 각 층은 다음과 같은 두 가지 레이어로 구성된다.

1. 멀티헤드 어텐션 레이어: 입력 시퀀스 내에서 각 토큰의 관계를 학습하기 위해 사용된다. 벡터를 이용해서 어텐션을 수행한다. *멀티 헤드 어텐션 레이어는 여러 개의 어텐션 헤드로 구성되어 있으며*, 이를 통해 다양한 과점에서 입력 시퀀스 내의 관계를 학습할 수 있다. 

2. 피드포워드 신경망 레이어: 멀티 헤드 어텐션 레이어를 통해 얻은 정보를 바탕으로 입력 시퀀스 내의 패턴을 학습하기 위해 사용된다. 이 레이어에서는 입력 벡터를 선형 변환하여 비선형 활성화 함수를 통해 처리한 뒤 다시 선형 변환하여 출력 벡터를 계산한다.


인코더에서 출력된 시퀀스는 디코더에서 입력으로 사용되며, 디코더는 이를 바탕으로 출력 시퀀스를 생성한다.

*인코더의 셀프 어텐션*

인코더에서 사용되는 멀티 헤드 어텐션 레이어는 ***셀프 어텐션***을 기반으로 한다.

셀프 어텐션은 입력 시퀀스 내의 각 토큰이 다른 토큰들과 어떤 관계를 가지는지를 학습하는 방법이다. 

입력 시퀀스의 각 토큰마다 **서로 다른 어텐션 헤드를 사용하여 다른 관점에서의 어텐션을 수행한다.** 이를 통해 입력 시퀀스 내의 다양한 학습을 할 수 있다.

*스케일드 점곱 어텐션* -> 어텐션 메커니즘 중 하나

스케일드 점곱 어텐션은 쿼리벡터와 키벡터의 내적 결과를 스케일링한 값에 소프트 맥스 함수를 적용하여 각각의 밸류벡터와 가중합을 구한다.

아래 코드는 스케일드 점곱 어텐션이다

In [11]:
#텍스트 토큰화
from transformers import AutoTokenizer
model_ckpt = "bert-base-uncased"
text = "time flies like an arrow"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [10]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [12]:
#밀집 임베딩 생성
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

#30522개의 정수 인덱스를 768 차원의 벡터로 매핑

Embedding(30522, 768)

In [14]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()

#[batch_size, seq_len, hidden_dim]

torch.Size([1, 5, 768])

In [15]:
import torch
from math import sqrt

query = key = value = inputs_embeds #쿼리,키,값 벡터 제작
dim_k = key.size(-1) # 768

#행렬 곱 계산
# [batch_size,m,n] X [batch_size,n,p] 모양일 때만 곱할 수 있다

scores = torch.bmm(query,key.transpose(1,2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

In [16]:
#소프트 맥스 함수 적용
#각 행의 모든 열값을 더하면 1이 됨
import torch.nn.functional as F

weights = F.softmax(scores, dim = -1)
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

In [17]:
#값(value)에 가중치 적용
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [ ]:
#하나의 함수로 표현
def scaled_dot_product_attention(query,key,value):
  dim_k = query.size(-1)
  scores = torch.bmm(query,key.transpose(1,2)) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  
  return torch.bmm(weights,value)

**인코더: 멀티헤드 어텐션 구현**

In [18]:
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    #투영
    super().__init__()
    self.q = nn.Linaer(embed_dim,head_dim)
    self.k = nn.Linaer(embed_dim,head_dim)
    self.v = nn.Linaer(embed_dim,head_dim)

  def forward(self, hidden_state):
    attn_outputs = scaled_dot_product_attention(self.q(hidden_state),self.k(hidden_state),self.v(hidden_state))
    return attn_outputs


In [19]:
class MultiHeadAttention(nn.Module):
  #head_dim = 64 
  # embed_dim // num_heads == 768 /12 == 64
  def __init__(self, config):
    super.__init__()
    embed_dim = config.hidden_size
    num_heads = config.num_attention_heads
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList([AttentionHead(embed_dim,head_dim) for _ in range(num_heads)])
    self.output_linaer = nn.Linear(embed_dim,embed_dim)

  def forward(self, hidden_state):
    x = torch.cat([h(hidden_state) for h in self.heads], dim = -1)
    #마지막 차원 축으로 연결
    x = self.output_linear(x)
    return x

In [20]:
#피드포워드 층
class FeedForward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu == nn.GELU()
    self.dropout = nn.Dropout(config.hidden_droupout_prob)

  def forward(self,x):
    x = self.linear_1(x)
    x = self.gelu(x)
    x = self.linear_2(x)
    x = self.dropout(x)
    return x

#토큰의 위치에 상관없이 모든 토큰 임베딩에 병렬적으로 적용됨

#GELU : 활성화 함수

**인코더: 층 정규화**

배치에 있는 각 입력 벡터를 평균이 0이고 분산이 1이 되도록 정규화

* 사후 층 정규화
* 사전 층 정규화

**스킵 연결**

층을 통과시키지 않은 텐서를 층의 출력과 더하는 트리
 
error 의 축적 방지